# Preparació de dades

En aquest *notebook*, convertim les dades originals a formats que es poden treballar a *Python*.

In [4]:
import pandas as pd
import numpy as np 
import plydata as ply
import datetime
import yaml
from functools import lru_cache

ply.options.modify_input_data = True


In [5]:
original = pd.read_csv("data/input/20171101_actualitzat_llistat_Llei_11_2017.csv",delimiter=";",encoding="latin-1")

original.head()

,Codi,cognoms nom,cognoms,nom,sexe,edat,Municipi naixement,Pedanies/Agregats naixement,Comarca naixement,Província naixement,...,Tipus procediment 2,num causa,any inicial,any aprovació sen o altra resol,pena,commutació/indult(demanat),afusellades,ref num arxiu,autoria i data de la descripció,data correccio registre
0,103990,"""AJOBLANCO"" (Revista) \r\n(per la publicació d...",NaN,AJOBLANCO\r\n(per la publicació de la historie...,--,0.0,--,--,--,--,...,Diligències prèvies,000516/IV78,1978.0,1979.0,Arxiu,NaN,NaN,105526.0,ANC 2017 07 12,NaN
1,144537,"""AURORA, La"" (revista del PORE)",NaN,LA AURORA,--,0.0,--,--,--,--,...,Causa ordinària,000038/IV78,1978.0,1978.0,Sobreseïment,NaN,NaN,104676.0,ANC 2017 07 12,NaN
2,143742,"""BRAVOS, LOS""",NaN,LOS BRAVOS,--,0.0,--,--,--,--,...,Diligències prèvies,000334/IV68,1968.0,1969.0,Sense declaració de responsabilitats,NaN,NaN,96807.0,ANC 2017 07 12,NaN
3,134935,"""CENTRO DE DEPORTES VILLANUEVA""",NaN,CENTRO DE DEPORTES VILLANUEVA,--,0.0,--,--,--,--,...,Diligències prèvies,--,1940.0,1942.0,Sense declaració de responsabilitats,NaN,NaN,43298.0,ANC 2017 07 12,NaN
4,144051,"""COOPERATIVA AGROPECUARIA DE GUISSONA. CASA RU...",NaN,COOPERATIVA AGROPECUARIA DE GUISSONA CASA RURA...,--,0.0,--,--,--,--,...,Diligències prèvies,000272/IV71,1971.0,1971.0,Sobreseïment,NaN,NaN,98248.0,ANC 2017 07 12,NaN


## Canviar noms de columnes a identificadors permesos en *Python*

In [7]:
original >> ply.rename(
{'afusellades': 'afusellades',
 'any_inicial': 'any inicial',
 'any_resol': 'any aprovació sen o altra resol',
 'ca_naix': 'Comunitat autònoma naixement',
 'ca_res': 'Comunitat autònoma residència',
 'codi': 'Codi',
 'cognoms': 'cognoms',
 'cognoms_nom': 'cognoms nom',
 'com_naix': 'Comarca naixement',
 'com_res': 'Comarca residència',
 'correccio': 'data correccio registre',
 'descr': 'autoria i data de la descripció',
 'edat': 'edat',
 'indult': 'commutació/indult(demanat)',
 'mun_naix': 'Municipi naixement',
 'mun_res': 'Municipi residència',
 'nom': 'nom',
 'num_causa': 'num causa',
 'pais_naix': 'País naixement',
 'pais_res': 'País residència',
 'ped_naix': 'Pedanies/Agregats naixement',
 'ped_res': 'Pedanies/Agregats residencia',
 'pena': 'pena',
 'proc_1': 'Tipus procediment 1',
 'proc_2': 'Tipus procediment 2',
 'prov_naix': 'Província naixement',
 'prov_res': 'Província residència',
 'ref_arxiu': 'ref num arxiu',
 'sexe': 'sexe'}
)

original.head()

,codi,cognoms_nom,cognoms,nom,sexe,edat,mun_naix,ped_naix,com_naix,prov_naix,...,proc_2,num_causa,any_inicial,any_resol,pena,indult,afusellades,ref_arxiu,descr,correccio
0,103990,"""AJOBLANCO"" (Revista) \r\n(per la publicació d...",NaN,AJOBLANCO\r\n(per la publicació de la historie...,--,0.0,--,--,--,--,...,Diligències prèvies,000516/IV78,1978.0,1979.0,Arxiu,NaN,NaN,105526.0,ANC 2017 07 12,NaN
1,144537,"""AURORA, La"" (revista del PORE)",NaN,LA AURORA,--,0.0,--,--,--,--,...,Causa ordinària,000038/IV78,1978.0,1978.0,Sobreseïment,NaN,NaN,104676.0,ANC 2017 07 12,NaN
2,143742,"""BRAVOS, LOS""",NaN,LOS BRAVOS,--,0.0,--,--,--,--,...,Diligències prèvies,000334/IV68,1968.0,1969.0,Sense declaració de responsabilitats,NaN,NaN,96807.0,ANC 2017 07 12,NaN
3,134935,"""CENTRO DE DEPORTES VILLANUEVA""",NaN,CENTRO DE DEPORTES VILLANUEVA,--,0.0,--,--,--,--,...,Diligències prèvies,--,1940.0,1942.0,Sense declaració de responsabilitats,NaN,NaN,43298.0,ANC 2017 07 12,NaN
4,144051,"""COOPERATIVA AGROPECUARIA DE GUISSONA. CASA RU...",NaN,COOPERATIVA AGROPECUARIA DE GUISSONA CASA RURA...,--,0.0,--,--,--,--,...,Diligències prèvies,000272/IV71,1971.0,1971.0,Sobreseïment,NaN,NaN,98248.0,ANC 2017 07 12,NaN


## Canviar números d'anys a dates per automatitzar el càlcul dels eixos 


In [8]:
# l'ús de lru_cache fa tot més ràpid, perquè cada any només ha de ser convertit a data una sola vegada


@lru_cache(maxsize=100)
def pd_year(n: float) :
    try :
        return pd.date_range(start=datetime.datetime(int(n),1,1),end=datetime.datetime(int(n),12,31),freq="Y")[0]
    except (TypeError,ValueError,OverflowError) :
        return n


original["any_inicial"] = original["any_inicial"].apply(pd_year)
original["any_resol"] = original["any_resol"].apply(pd_year)
original.loc[:4,["cognoms_nom","any_inicial","any_resol"]]

,cognoms_nom,any_inicial,any_resol
0,"""AJOBLANCO"" (Revista) \r\n(per la publicació d...",1978-12-31,1979-12-31
1,"""AURORA, La"" (revista del PORE)",1978-12-31,1978-12-31
2,"""BRAVOS, LOS""",1968-12-31,1969-12-31
3,"""CENTRO DE DEPORTES VILLANUEVA""",1940-12-31,1942-12-31
4,"""COOPERATIVA AGROPECUARIA DE GUISSONA. CASA RU...",1971-12-31,1971-12-31


## Categoritzar penes

Aquestes categories són derivades del [*Anexo de la Circular de Creación de las Comisiones de Examen de Penas* del any 1940](https://www.boe.es/datos/pdfs/BOE//1940/026/A00662-00665.pd) 

| categoria | calificació | |
| ----------| ----------- | --- |
|1	|pena de mot	||	
|2	|30 anys i un dia a perpètua||		
|3	|20 anys i un dia a 30 anys |Reclusión	Mayor|
|4	|12 anys i un día a 20 anys|Menor|
|5	|6 anys i un dia a 12 anys |Prisión	Mayor|
|6	|6 mesos i un dia a 6 anys		|Menor|
|7	|Fins 6 mesos	||
|8	|sin pena		||
|9	|altres		||

In [9]:
categories_penes={ a["pena"] : a  for a in yaml.load(open("data/input/categories.yml")) }

original["pena_cat"] = original["pena"].apply(
    lambda a: categories_penes.get(a,{ 'categoria' : None })['categoria'])

original["pena_cat"].value_counts()

8.0    30199
4.0    19791
1.0     4417
2.0     3747
9.0     3632
5.0     2968
6.0     1768
3.0     1205
7.0     1013
Name: pena_cat, dtype: int64

## Categoritzar origen 

Categories: 

  - cc - resident i nascut a Catalunya fora da província de Barcelona
  - bb - resident i nascut a la província de Barcelona
  - be - resident a la província de Barcelona i nascut a Espanya fora da província de Barcelona
  - ce - resident a Catalunya fora de la província de Barcelona i nascut a Espanya fora de Catalunya
  - ee - resident i nascut a Espanya fora de Catalunya
  - i  - nascut fora de Espanya
  - ? - província o país de residència no consta 
  - cb - resident a Catalunya fora de la província de Barcelona i nascut a la província de Barcelona
  - ec - resident a Espanya fora de Catalunya i nascut a Catalunya
  - \- sense domicíli fix
  

In [39]:
def cat_origen(cas) :
    if cas.prov_res == '--' or cas.pais_res == '-':
        return "?"
    if cas.prov_res == "Barcelona" :
        res="b"
    elif cas.ca_res == "Catalunya" :
        res="c"
    elif cas.pais_res == "Espanya" :
        res="e"
    elif cas.pais_res.index("Sense domicili fix")>-1 :
        res="-"
    else :
        res="i"
    if cas.prov_naix == "Barcelona" :
        nac="b"
    elif cas.ca_naix == "Catalunya" :
        nac="c"
    elif cas.pais_naix == "Espanya" :
        nac="e"
    else:
        nac="i"
    if nac == 'i' :
        return nac
    if res == '-' :
        return res
    if res == 'e' and nac in ('c','b','e') :
        return 'e'
    if res == 'b' and nac in ('c','e') :
        return 'be'
    return res+nac
 


original["origen_cat"]=original.apply(cat_origen,axis=1)
original["origen_cat"].value_counts()

cc    21704
be    19391
bb    19241
ce     2732
i      2652
e      2285
?       946
cb      843
-        10
Name: origen_cat, dtype: int64

In [33]:
original.to_msgpack("data/processat.msg")

In [40]:
original.query("origen_cat == '-'").loc[:,["cognoms_nom","pais_naix","pais_res"]]

,cognoms_nom,pais_naix,pais_res
11879,"CAPARRÓS RODRÍGUEZ, Alfonso",Espanya,Espanya / Sense domicili fix
13478,"CASAS MARCEL, Antonio",Espanya,Espanya / Sense domicili fix
17842,"CRUZ EXPÓSITO, Francisco de la",Espanya,Sense domicili fix
19469,"DORADO LÓPEZ, Joaquín",Espanya,Sense domicili fix
22208,"FERNÁNDEZ ÁLVAREZ, Ramón",Espanya,Sense domicili fix
26291,"GARCÍA MARTÍNEZ, Florencio",Espanya,Sense domicili fix
31749,"IBORRA RIUS, Antonio",Espanya,Espanya / Sense domicili fix
36888,"MARÍN RUIZ, Pablo",Espanya,Sense domicili fix
58685,"SÁNCHEZ GONZÁLEZ, Ángel",Espanya,Sense domicili fix
64410,"TORIBIO ESCRIBANO, José",Espanya,Sense domicili fix
